# 파이썬을 활용한 머신러닝
## 스피어 피싱 모의훈련 서비스 제작

<strong>idea : </strong>
1. 이 프로젝트의 주제는 개인에 대한 스피어피싱 문구 제작이다.
2. 현재는 키워드를 입력하면 그에 맞게 데이터를 수집해 데이터프레임을 생성하고 현재 엑셀파일로 변환까지 한다.
3. 2번대로만 코드가 수행된다면 1번의 내용을 이행하지 못한다.
4. 3번대로 1번의 내용을 수행하기 위해서는 엑셀 파일에 여러 데이터 시트가 있는 점을 활용해 공격하고자 하는 유사한 키워드를 모아서 수집한걸 데이터프레임에 각 저장 후 엑셀의 여러 시트에 기록한다. 
5. 이후 자연어 처리는 엑셀의 여러 시트를 참조하여 공격하고자 하는 대상의 아이디를 탐색해 여러 정보들을 머신러닝으로 파악하고자 한다.

<strong>자연어 처리 계획 : </strong>
1. 자연어 처리는 일단 여러 시트를 참조하기 위한 data_set 엑셀 파일과 공격하는 대상에 대한 정보가 저장되는 파일을 새로 만든다,
2. data_set 엑셀 파일에서 공격대상에 대한 여러 테이블 정의서에 예상한대로 여러 정보가 저장되는 새 엑셀 파일 기반 자연어 처리를 할 것이다. 자연어 처리에서는 다음의 정보들을 머신러닝으로 파악하고자 한다.

<strong>자연어 처리에서 파악하고자 하는 정보: </strong> nltk 
1. 많이 사용하는 단어, 어투
2. 특정 단어로 시작하는 그룹

<strong>결과물 : </strong>
1. 머신러닝을 돌려 사람의 성향을 파악했네? (tensorflow or sklearn)
2. 성향에 맞는 문구를 작성해 그 사람이 특정 링크 혹 파일을 누르도록 유도. (nltk)
3. 모의훈련이니 모의훈련에 맞는 모의훈련 결과 그래프 그리기(seaborn)

<strong>주의 : </strong>
+ ^^ 중요한건 기간이 짧다.
+ ??? : 일어나 코딩해야지 *^0^*.

### 모듈_호출

In [1]:
from bs4 import BeautifulSoup #웹 크롤링
from urllib.request import urlopen #url
from urllib import parse #url2
import tweepy #트위터
import numpy as np #기본
import pandas as pd #데이터프레임
import matplotlib.pyplot as plt #그래프
import nltk #자연어처리

### 키워드

In [2]:
keyword = ['아이돌','여자친구','방탄소년단','엑소','하스스톤','백현']
results = []
for x in range(len(keyword)):
    point = parse.quote(keyword[x])
    results.append("https://twitter.com/search?q="+point+"&src=typed_query&lang=ko")
    print(results[x],end="\n")

print("\n")
urlp = results[1]
print(urlp)

https://twitter.com/search?q=%EC%95%84%EC%9D%B4%EB%8F%8C&src=typed_query&lang=ko
https://twitter.com/search?q=%EC%97%AC%EC%9E%90%EC%B9%9C%EA%B5%AC&src=typed_query&lang=ko
https://twitter.com/search?q=%EB%B0%A9%ED%83%84%EC%86%8C%EB%85%84%EB%8B%A8&src=typed_query&lang=ko
https://twitter.com/search?q=%EC%97%91%EC%86%8C&src=typed_query&lang=ko
https://twitter.com/search?q=%ED%95%98%EC%8A%A4%EC%8A%A4%ED%86%A4&src=typed_query&lang=ko
https://twitter.com/search?q=%EB%B0%B1%ED%98%84&src=typed_query&lang=ko


https://twitter.com/search?q=%EC%97%AC%EC%9E%90%EC%B9%9C%EA%B5%AC&src=typed_query&lang=ko


### 트위터_수집

In [3]:
consumer_key = "oCfme0qMTyYybPUU0ABwKNHKg"
consumer_secret = "1VP4phAtG0Y1ILTe5RGL7XYGHkNrWEzzxv8fk0cWym1p9yfNIO"
access_token = "2910321932-RkciOAbw8WYB2nIISDrFSREfbDnxDIydAdX2mLB"
access_token_secret = "faRZmsdWYWQ8mLdYN40qmj7B5eA0Ifl9RglLohzi8fCqa"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret) 
api = tweepy.API(auth) 

results = []

for tweet in tweepy.Cursor(api.search, q=keyword[5], count=10).items():
    results.append(tweet)
    
print(len(results))
#print(results[0])

342


### 데이터프레임

In [4]:
def process_results(results):
    id_list = [tweet.id for tweet in results]
    data_set = pd.DataFrame(id_list, columns=["id"])

    # Processing Tweet Data
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in results]
    data_set["source"] = [tweet.source for tweet in results]

    # Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_name"] = [tweet.author.name for tweet in results]
    data_set["user_created_at"] = [tweet.author.created_at for tweet in results]
    data_set["user_description"] = [tweet.author.description for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]

    return data_set

data_set = process_results(results)
data_set

,id,text,created_at,retweet_count,favorite_count,source,user_id,user_screen_name,user_name,user_created_at,user_description,user_followers_count,user_friends_count,user_location
0,1126091695227957250,RT @DreamMate_0104: #BBMAs \n#BBMAsTopSocial E...,2019-05-08 11:49:16,283,0,Twitter for Android,1125900089304567808,bbhhtbq,Quyên,2019-05-07 23:07:54,BBH ~❤,0,280,Việt Nam
1,1126091687934042112,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:49:14,1877,0,Twitter for Android,911777474517671936,younaexo27,에리유나,2017-09-24 02:21:04,❤다시 태어나도 엑소엘 ❤ 무조건 9인지지\n 멘션 스...,846,956,
2,1126091663107952640,RT @Bbhgap: #PremiosMTVMIAW #MTVLAKPOPEXO #MTV...,2019-05-08 11:49:08,440,0,Twitter for Android,1011153858872000512,04Pubinba18,도유,2018-06-25 07:47:21,9인지지💕 | 04 엑소엘 | n인지지 안받아요 | 9명모두가 최애😍 | 편하게 다...,146,161,
3,1126091638160232449,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:49:02,1877,0,Twitter for Android,936526752737722368,matchahyunnie,𝐰𝐧²✨,2017-12-01 09:25:52,"【𝐁𝐞 𝐁𝐫𝐚𝐯𝐞, 𝐁𝐞 𝐇𝐮𝐦𝐛𝐥𝐞】\n中文 🈵 ENG 🆗️\n ...",233,271,☁ c l o u d 💫
4,1126091620833619969,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:48:58,1877,0,Twitter for iPhone,1066627727153815552,dyun_EXO,듄듄🐾,2018-11-25 09:40:41,Don't mess up my tempo,192,199,EXOplanet
5,1126091612184928256,RT @shune_0408: Q.백현이가 싫어하는 야채는?\n#PremiosMTVM...,2019-05-08 11:48:56,1159,0,Twitter for iPhone,1006741431086510080,0326_DAY,🦋 데이,2018-06-13 03:33:56,🦋매일 매일 시우민데이🦋,44,141,
6,1126091582610890752,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:48:49,1877,0,Twitter for iPhone,1109262175581790208,jjung_0326,[반휴트]우시쟘❄️,2019-03-23 01:14:46,"n인지지, 공출목❌",55,62,
7,1126091552046993408,RT @BBH_56_4: 여러분 (오늘 백현 생일인 거) 아시져~♡\n\n#함께걷는...,2019-05-08 11:48:42,72,0,Twitter for Android,2572790185,BBHJ506,구구백,2014-06-17 11:51:36,같이 노력하면 되죠. 나도 노력하는 건데 뭐. @B_hundred_Hyun,73,189,배켜니마음속
8,1126091522598895617,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:48:35,1877,0,Twitter for Android,1065283260434001920,Taj241,Taj__61,2018-11-21 16:38:16,شامية من بلاد الياسمين \nنازلة بمكوك فضائي من ...,129,157,
9,1126091496078188545,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:48:29,1877,0,Twitter for Android,880826288134537216,windlight0727,달보드레,2017-06-30 16:32:06,,17,1202,


### 입출력

In [5]:
#index = str(1)
#hello = 'Sheet' + index
data_set.to_excel('data_set.xlsx',sheet_name='Sheet1')
data_set = pd.read_excel('data_set.xlsx', sheetname='Sheet1')
data_set

C:\Users\dsz08\Anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


,Unnamed: 0,id,text,created_at,retweet_count,favorite_count,source,user_id,user_screen_name,user_name,user_created_at,user_description,user_followers_count,user_friends_count,user_location
0,0,1126091695227956992,RT @DreamMate_0104: #BBMAs \n#BBMAsTopSocial E...,2019-05-08 11:49:16,283,0,Twitter for Android,1125900089304568064,bbhhtbq,Quyên,2019-05-07 23:07:54,BBH ~❤,0,280,Việt Nam
1,1,1126091687934041984,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:49:14,1877,0,Twitter for Android,911777474517671936,younaexo27,에리유나,2017-09-24 02:21:04,❤다시 태어나도 엑소엘 ❤ 무조건 9인지지\n 멘션 스...,846,956,NaN
2,2,1126091663107953024,RT @Bbhgap: #PremiosMTVMIAW #MTVLAKPOPEXO #MTV...,2019-05-08 11:49:08,440,0,Twitter for Android,1011153858872001024,04Pubinba18,도유,2018-06-25 07:47:21,9인지지💕 | 04 엑소엘 | n인지지 안받아요 | 9명모두가 최애😍 | 편하게 다...,146,161,NaN
3,3,1126091638160231936,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:49:02,1877,0,Twitter for Android,936526752737722368,matchahyunnie,𝐰𝐧²✨,2017-12-01 09:25:52,"【𝐁𝐞 𝐁𝐫𝐚𝐯𝐞, 𝐁𝐞 𝐇𝐮𝐦𝐛𝐥𝐞】\n中文 🈵 ENG 🆗️\n ...",233,271,☁ c l o u d 💫
4,4,1126091620833619968,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:48:58,1877,0,Twitter for iPhone,1066627727153816064,dyun_EXO,듄듄🐾,2018-11-25 09:40:41,Don't mess up my tempo,192,199,EXOplanet
5,5,1126091612184928000,RT @shune_0408: Q.백현이가 싫어하는 야채는?\n#PremiosMTVM...,2019-05-08 11:48:56,1159,0,Twitter for iPhone,1006741431086509952,0326_DAY,🦋 데이,2018-06-13 03:33:56,🦋매일 매일 시우민데이🦋,44,141,NaN
6,6,1126091582610891008,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:48:49,1877,0,Twitter for iPhone,1109262175581789952,jjung_0326,[반휴트]우시쟘❄️,2019-03-23 01:14:46,"n인지지, 공출목❌",55,62,NaN
7,7,1126091552046993024,RT @BBH_56_4: 여러분 (오늘 백현 생일인 거) 아시져~♡\n\n#함께걷는...,2019-05-08 11:48:42,72,0,Twitter for Android,2572790185,BBHJ506,구구백,2014-06-17 11:51:36,같이 노력하면 되죠. 나도 노력하는 건데 뭐. @B_hundred_Hyun,73,189,배켜니마음속
8,8,1126091522598896000,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:48:35,1877,0,Twitter for Android,1065283260434002048,Taj241,Taj__61,2018-11-21 16:38:16,شامية من بلاد الياسمين \nنازلة بمكوك فضائي من ...,129,157,NaN
9,9,1126091496078189056,RT @SMmakesitLive: 🌟!t Live(잇라이브)🌟\n: The 12th...,2019-05-08 11:48:29,1877,0,Twitter for Android,880826288134537216,windlight0727,달보드레,2017-06-30 16:32:06,NaN,17,1202,NaN


### 자연어_처리